# 02. Расчет MDE и Размера Выборки

В этом ноутбуке мы рассчитаем:
- Минимальный обнаруживаемый эффект (MDE)
- Необходимый размер выборки для статистически значимого результата
- Проверим достаточность имеющихся данных

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import warnings
warnings.filterwarnings('ignore')

## 1. Загрузка Данных и Подготовка

In [ ]:
# Загружаем итоговые результаты анализа
df = pd.read_excel('../data/processed/final_results_to_analyze.xlsx')

print("Структура данных:")
print(df.info())
print("\nПервые 5 строк:")
print(df.head())
print(f"\nРазмер датасета: {df.shape}")

## 2. Расчет Базовых Метрик по Группам

In [ ]:
# Разделяем данные по группам
control = df[df['ab_group'] == 'control']
test = df[df['ab_group'] == 'test']

print(f"Контрольная группа: {control.shape[0]:,} пользователей")
print(f"Тестовая группа: {test.shape[0]:,} пользователей")

# Рассчитываем базовую CR в контрольной группе
control_users_with_views = control['cnt_users_ads'].sum()
control_users_with_adds = control[control['cnt_adds_ads'] > 0]['cnt_users_ads'].sum()
baseline_cr = (control_users_with_adds / control_users_with_views) * 100

print(f"\nБазовая CR (Control): {baseline_cr:.2f}%")
print(f"Пользователей с просмотром (Control): {control_users_with_views:,}")
print(f"Пользователей с добавлением (Control): {control_users_with_adds:,}")

## 3. Параметры Статистического Теста

In [ ]:
# Параметры A/B теста
alpha = 0.05  # Уровень значимости (риск ошибки первого рода)
power = 0.80  # Статистическая мощность (1 - beta)
beta = 1 - power

# z-значения для нормального распределения
z_alpha = stats.norm.ppf(1 - alpha/2)  # Двусторонний тест
z_beta = stats.norm.ppf(power)

print(f"Параметры теста:")
print(f"  Уровень значимости (α): {alpha}")
print(f"  Статистическая мощность: {power}")
print(f"  z-значение для α: {z_alpha:.4f}")
print(f"  z-значение для β: {z_beta:.4f}")

## 4. Расчет MDE (Minimum Detectable Effect)

In [ ]:
# Функция для расчета MDE
def calculate_mde(baseline_rate, sample_size_per_group, alpha=0.05, power=0.80):
    """
    Рассчитывает MDE для двоичной переменной (конверсия)
    
    Аргументы:
        baseline_rate: Базовая конверсия в контрольной группе (доля, 0-1)
        sample_size_per_group: Размер выборки на одну группу
        alpha: Уровень значимости
        power: Статистическая мощность
    
    Возвращает:
        MDE в абсолютных пунктах и процентах
    """
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    # Стандартная ошибка
    se = np.sqrt(baseline_rate * (1 - baseline_rate) / sample_size_per_group)
    
    # MDE в абсолютных пунктах
    mde_absolute = (z_alpha + z_beta) * se * 2
    
    # MDE в процентах (от базовой величины)
    mde_relative = (mde_absolute / baseline_rate) * 100
    
    return mde_absolute, mde_relative

# Получаем текущие размеры групп
current_sample_size = control.shape[0]  # Предполагаем равные размеры групп

# Рассчитываем MDE
baseline_rate = baseline_cr / 100
mde_absolute, mde_relative = calculate_mde(
    baseline_rate=baseline_rate,
    sample_size_per_group=current_sample_size,
    alpha=alpha,
    power=power
)

print(f"Текущий размер выборки на группу: {current_sample_size:,}")
print(f"\nМинимально обнаруживаемый эффект (MDE):")
print(f"  Абсолютный: {mde_absolute*100:.2f} процентных пункта")
print(f"  Относительный: {mde_relative:.2f}%")
print(f"\nЭто означает:")
print(f"  Минимальная CR в Test: {(baseline_rate + mde_absolute)*100:.2f}%")
print(f"  Это {mde_relative:.2f}% выше базовой CR")

## 5. Расчет Требуемого Размера Выборки

In [ ]:
# Функция для расчета необходимого размера выборки
def calculate_sample_size(baseline_rate, target_effect_relative, alpha=0.05, power=0.80):
    """
    Рассчитывает необходимый размер выборки для обнаружения эффекта
    
    Аргументы:
        baseline_rate: Базовая конверсия (доля, 0-1)
        target_effect_relative: Ожидаемый относительный эффект (%)
        alpha: Уровень значимости
        power: Статистическая мощность
    
    Возвращает:
        Требуемый размер выборки на группу
    """
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    # Целевая конверсия
    target_rate = baseline_rate * (1 + target_effect_relative/100)
    
    # Объединенная вероятность
    p_pool = (baseline_rate + target_rate) / 2
    
    # Формула для расчета размера выборки
    n = 2 * ((z_alpha + z_beta)**2) * p_pool * (1 - p_pool) / (baseline_rate - target_rate)**2
    
    return int(np.ceil(n))

# Рассчитываем размер выборки для разных целевых эффектов
effects = [1.0, 2.0, 2.87, 3.0, 5.0]
print("Требуемый размер выборки для разных целевых эффектов:")
print("\nОтносительный эффект | Требуемый размер (на группу) | Всего пользователей")
print("-" * 70)

for effect in effects:
    required_n = calculate_sample_size(
        baseline_rate=baseline_rate,
        target_effect_relative=effect,
        alpha=alpha,
        power=power
    )
    total_users = required_n * 2
    print(f"{effect:>18.2f}% | {required_n:>28,} | {total_users:>19,}")

# Проверяем конкретный целевой эффект
target_effect = 2.87
required_sample_size = calculate_sample_size(
    baseline_rate=baseline_rate,
    target_effect_relative=target_effect,
    alpha=alpha,
    power=power
)
total_required = required_sample_size * 2

print(f"\n" + "="*70)
print(f"Для обнаружения эффекта +{target_effect}%:")
print(f"  Требуется: {required_sample_size:,} пользователей на группу")
print(f"  Всего: {total_required:,} пользователей")
print(f"  Текущая выборка: {current_sample_size:,} на группу ({current_sample_size*2:,} всего)")
print(f"  Статус: {'✅ ДОСТАТОЧНО' if current_sample_size >= required_sample_size else '❌ НЕ ДОСТАТОЧНО'}")

## 6. Анализ Статистической Мощности

In [ ]:
# Функция для расчета статистической мощности
def calculate_power(baseline_rate, target_effect_relative, sample_size_per_group, alpha=0.05):
    """
    Рассчитывает статистическую мощность для заданных параметров
    """
    z_alpha = stats.norm.ppf(1 - alpha/2)
    
    # Целевая конверсия
    target_rate = baseline_rate * (1 + target_effect_relative/100)
    
    # Объединенная вероятность
    p_pool = (baseline_rate + target_rate) / 2
    
    # Стандартная ошибка
    se = np.sqrt(2 * p_pool * (1 - p_pool) / sample_size_per_group)
    
    # z-значение для разницы
    z_effect = abs(target_rate - baseline_rate) / se
    
    # Мощность = P(Z > z_alpha - z_effect)
    power = stats.norm.sf(z_alpha - z_effect)
    
    return power

# Рассчитываем мощность при текущем размере выборки
actual_power = calculate_power(
    baseline_rate=baseline_rate,
    target_effect_relative=target_effect,
    sample_size_per_group=current_sample_size,
    alpha=alpha
)

print(f"Статистическая мощность при текущем размере выборки:")
print(f"  Размер выборки на группу: {current_sample_size:,}")
print(f"  Ожидаемый эффект: +{target_effect}%")
print(f"  Фактическая мощность: {actual_power*100:.2f}%")
print(f"  Целевая мощность: {power*100:.0f}%")
print(f"  Статус: {'✅ ДОСТАТОЧНО' if actual_power >= power else '❌ НЕДОСТАТОЧНО'}")

## 7. Итоговая Таблица Расчетов

In [ ]:
# Создаем итоговую таблицу
summary_data = {
    'Параметр': [
        'Базовая CR (Control)',
        'Целевая CR (Test)',
        'Относительный эффект',
        'Абсолютный эффект',
        'Уровень значимости (α)',
        'Статистическая мощность (1-β)',
        'Требуемый размер выборки (на группу)',
        'Текущий размер выборки (на группу)',
        'Статус выборки'
    ],
    'Значение': [
        f'{baseline_cr:.2f}%',
        f'{baseline_cr + mde_absolute*100:.2f}%',
        f'+{target_effect}%',
        f'+{mde_absolute*100:.2f}pp',
        f'{alpha:.0%}',
        f'{power:.0%}',
        f'{required_sample_size:,}',
        f'{current_sample_size:,}',
        '✅ ДОСТАТОЧНО' if current_sample_size >= required_sample_size else '❌ НЕ ДОСТАТОЧНО'
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\nИТОГОВА ТАБЛИЦА РАСЧЕТОВ")
print("="*70)
print(summary_df.to_string(index=False))

# Сохраняем результаты
summary_df.to_csv('../reports/mde_calculation_summary.csv', index=False)
print("\n✅ Результаты сохранены в: ../reports/mde_calculation_summary.csv")